<a href="https://colab.research.google.com/github/SasankBVM/GENERATIVE-AI/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers[torch] accelerate>=0.20.1
import transformers
import datasets
import accelerate
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns,math
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import classification_report,confusion_matrix
from transformers import AutoTokenizer,AutoModel,BertTokenizer,BertModel,DataCollatorWithPadding,AutoModelForSequenceClassification,Trainer
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

max_len=512
batch_sz=16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset=load_dataset("wiki_qa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 6165
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 2733
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 20360
    })
})

In [4]:
id2lab={0:"POSITIVE",1:"NEGATIVE"}
lab2id={"POSITIVE":1,"NEGATIVE":0}

In [5]:
checkpoint="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained("gpt2",num_labels=2,id2label=id2lab,label2id=lab2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
train_df=dataset["train"].to_pandas()
train_df=train_df[["answer","label"]]
val_df=dataset["validation"].to_pandas()
val_df=val_df[["answer","label"]]

In [7]:
def get_max_pad(data):
  maxi=0
  for i in range(len(data["answer"])):
    maxi=max(maxi,len(data["answer"].iloc[i]))
  return maxi
print("THE MAXIMUM LENGTH OF STRING THAT WE GET:",get_max_pad(train_df))

THE MAXIMUM LENGTH OF STRING THAT WE GET: 1075


In [8]:
def tokenize(data):
    return tokenizer(data["answer"], truncation=True, max_length=1075, padding='max_length', return_tensors="pt")

In [10]:
train_df["label"].value_counts()

0    19320
1     1040
Name: label, dtype: int64

In [11]:
class_weights = (1 - train_df["label"].value_counts().sort_index() / len(train_df)).values
class_weights = torch.from_numpy(class_weights)

In [34]:
tokens = train_df.apply(tokenize,axis=1)
val_tok=val_df.apply(tokenize,axis=1)
# train_encodings = {key: val.squeeze() for key, val in tokens.items()}
train_labels = torch.tensor(train_df["label"].to_numpy())
val_labels=torch.tensor(val_df["label"].to_numpy())

tensor([0, 0, 0,  ..., 0, 0, 0])

In [13]:
columns=["answer","labels"]
train_df.columns=columns

In [35]:
class LossTrainer(Trainer):
    def loss_funcs(self, model, inputs, return_op=False):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_op else loss

In [36]:
from sklearn.metrics import f1_score
def perf(pred):
  lables=pred.label_ids
  preds=pred.predictions.argmax(-1)
  f1=f1_score(lables,preds,average="weighted")
  return {"f1":f1}

In [37]:
batch_size = 64
steps = len(dataset["train"])
output_dir = "./result"
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # weight_decay=0.01,
    evaluation_strategy="epoch",
    # logging_steps=steps,
)

In [39]:
trainer = LossTrainer(
    model=model,
    args=training_args,
    compute_metrics=perf,
    train_dataset=tokens,
    eval_dataset=val_tok,
    tokenizer=tokenizer
)

In [40]:
trainer.train()

IndexError: index out of range in self